In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [25]:
links = [
"https://aliffchannel.com/netizen/",
"https://aliffchannel.com/travel/",
"https://aliffchannel.com/teknologi/",
"https://aliffchannel.com/lifestyle/",
"https://aliffchannel.com/islamik/"]
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           'Referer':'https://aliffchannel.com/'} 


In [26]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('h3', attrs = {"class":"jeg_post_title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('h3', attrs = {"class":"jeg_post_title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [27]:
max_worker = 10


for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.text, "html.parser")
    nav = soup.find('div',attrs={'class':'jeg_navigation'})
    a = nav.find_all('a', attrs = {"class":"page_number"})
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


https://aliffchannel.com/netizen/
36


100%|██████████| 4/4 [00:05<00:00,  1.26s/it]


https://aliffchannel.com/travel/
20


100%|██████████| 2/2 [00:05<00:00,  2.53s/it]


https://aliffchannel.com/teknologi/
15


100%|██████████| 2/2 [00:06<00:00,  3.32s/it]


https://aliffchannel.com/lifestyle/
111


100%|██████████| 12/12 [00:40<00:00,  3.36s/it]


https://aliffchannel.com/islamik/
2


100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


In [28]:
with open(f'aliffchannel-link.json', 'a') as f:
    json.dump(hrefs, f)

In [ ]:
with open('aliffchannel-link.json') as fopen:
    hrefs = json.load(fopen)

In [35]:
len(hrefs)

1738

In [34]:
hrefs = list(set(hrefs))

In [36]:
for x in tqdm(hrefs):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)

    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', class_="jeg_post_title").text
        h = soup.find('div', class_="content-inner")
        p = [p.text for p in h.find_all("p") if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        continue


    data = {'url': x, 'headline': headline, 'content': p}


    with open('aliffchannel.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')

100%|██████████| 1738/1738 [47:03<00:00,  1.62s/it]
